# Recommandation de Films

## Objectif : 
Fournir des recommandations aux utilisateurs en fonction du contenu de leurs films préférés.


In [6]:
import requests
import bs4


from requests import get
from bs4 import BeautifulSoup 
from warnings import warn
from time import sleep
from random import randint
import numpy as np, pandas as pd
import numpy as np

from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics.pairwise import linear_kernel


## Web scraping 
Dans un premier temps, on va "scraper" des informations sur les meilleurs films du site allocine https://www.allocine.fr/film/meilleurs/ 

In [2]:
pages = np.arange(1, 12, 1)
title  = []
note = []
resume = []

for page in pages : 
    #print(page)
    def get_page_contents(page):
        page = requests.get('https://www.allocine.fr/film/meilleurs/' 
                        + '?page=' + str(page) , headers={"Accept-Language": "fr"})
        return bs4.BeautifulSoup(page.text, "html.parser")

    soup = get_page_contents(page)
    movies = soup.findAll('div', class_='card entity-card entity-card-list cf')
    titles = [movie.find('a').text for movie in movies]
    title = title + titles
    note = note +  [movie.find('span', 'stareval-note', text_attribute=False).text.strip() for movie in movies]
    resume  = resume +  [movie.find('div', 'content-txt', text_attribute=False).text.strip() for movie in movies]


In [3]:
films_dict = {'title': title, 'resume': resume, 'note': note }

films = pd.DataFrame(films_dict)
films.headad()   

,title,resume,note
0,Forrest Gump,"Quelques décennies d'histoire américaine, des ...","2,6"
1,La Liste de Schindler,Evocation des années de guerre d'Oskar Schindl...,"4,2"
2,La Ligne verte,"Paul Edgecomb, Gardien-chef du pénitencier de ...","2,8"
3,12 hommes en colère,"Lors d'un procès, un juré émet l'hypothèse que...","5,0"
4,Le Parrain,"En 1945, à New York, les Corleone sont une des...","4,6"


###  Traitement de text ( calcul TFIDF)

In [5]:
sw = list(fr_stop)
tfidf = TfidfVectorizer(stop_words=sw)
films['resume'] = films['resume'].fillna('')
tfidf_matrix = tfidf.fit_transform(films['resume'])
tfidf_matrix.shape


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['neuf', 'qu', 'quelqu'] not in stop_words.
  warnings.warn(


(110, 1624)

### Matrice de similarité

In [7]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)


In [8]:
indices = pd.Series(films.index, index=films['title']).drop_duplicates()


### Recommandation :

In [12]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:6]

    movie_indices = [i[0] for i in sim_scores]

    return films['title'].iloc[movie_indices]


In [17]:
#test
get_recommendations("Vol au-dessus d'un nid de coucou")


67                                     Shutter Island
35                                 Les Temps modernes
65                                             Matrix
54                          Les Sentiers de la gloire
20    Star Wars : Episode V - L'Empire contre-attaque
Name: title, dtype: object

In [18]:
rec = pd.DataFrame(get_recommendations("Vol au-dessus d'un nid de coucou")
)

In [19]:
rec

,title
67,Shutter Island
35,Les Temps modernes
65,Matrix
54,Les Sentiers de la gloire
20,Star Wars : Episode V - L'Empire contre-attaque


In [28]:
pd.merge(rec, films, left_index=True, right_index=True).sort_values("note",  ascending=False)


,title_x,title_y,resume,note
20,Star Wars : Episode V - L'Empire contre-attaque,Star Wars : Episode V - L'Empire contre-attaque,"Malgré la destruction de l'Etoile Noire, l'Emp...","5,0"
35,Les Temps modernes,Les Temps modernes,Charlot est ouvrier dans une immense usine. Il...,"4,6"
54,Les Sentiers de la gloire,Les Sentiers de la gloire,"Durant la Première Guerre mondiale, le général...","4,6"
67,Shutter Island,Shutter Island,1954. Teddy Daniels et Chuck Aule enquêtent su...,"3,8"
65,Matrix,Matrix,"Programmeur anonyme, Thomas Anderson est aussi...","3,4"
